In [1]:
import pandas as pd

In [6]:
apt_price = pd.read_csv(
    'apt_price_2022.csv',
    encoding='cp949',
    skiprows=15,
    thousands=','
)
# apt_price['시군구'] 를 시, 구, 동으로 분할
apt_price['구'] = apt_price['시군구'].str.split(' ').str[1]
apt_price['동'] = apt_price['시군구'].str.split(' ').str[2]
# 전용면적을 평 변환
apt_price['평'] = round(apt_price['전용면적(㎡)'] / 3.03,1)
# 거래금액 단위를 억원으로 변환
apt_price['거래금액_억원'] = apt_price['거래금액(만원)'] / 10000
# 계약년월 -> 계약년, 계약월로 분리
apt_price['계약년'] = apt_price['계약년월'] // 100
apt_price['계약월'] = apt_price['계약년월'] % 100
# 평정보를 이용하여 평형정보 생성
# 10평이하, 10평대, 20평대, 30평대, 40평대, 40평이상
apt_price['평형']=pd.cut(
    apt_price['평'],
    bins=[0,10,20,30,40,50,1000],
    labels='10평이하,10평대,20평대,30평대,40평대,40평이상'.split(',')
)
# 강남3구의 평형별 평균거래가와 그 외 지역 평형별 평균 거래가 비교
cond_매매 = apt_price['구'].isin(['강남구','송파구','서초구'])
강남3구_매매=apt_price[cond_매매].groupby('평형')[['거래금액_억원']].mean()
강남3구_매매.rename(columns={'거래금액_억원':'평균매매가'},inplace=True)
그외22구_매매=apt_price[~cond_매매].groupby('평형')[['거래금액_억원']].mean()
그외22구_매매.rename(columns={'거래금액_억원':'평균매매가'},inplace=True)

In [7]:
강남3구_매매

,평균매매가
평형,
10평이하,3.868691
10평대,10.448434
20평대,21.872192
30평대,27.289838
40평대,32.071870
40평이상,37.298094


In [ ]:
apt_rent = pd.read_csv('apt_rent.csv',encoding='cp949',skiprows=15,thousands=',')
# (50만원 x 12개월) / 4.75%(전월세전환율) = 약 126,315,000원에 기존 보증금 1,000만원을 더합니다.
deposit =((apt_rent['월세(만원)']*12)*0.0475 +apt_rent['보증금(만원)'])/10000
apt_rent['전세금_억원'] = round(deposit,2)
# 보증금 및 월세를 전세금으로 전환 후 단위를 억원으로 변환
deposit =((apt_rent['월세(만원)']*12)*0.0475 +apt_rent['보증금(만원)'])/10000
apt_rent['전세금_억원'] = round(deposit,2)
# apt_rent['시군구'] 를 시, 구, 동으로 분할
apt_rent['구'] = apt_rent['시군구'].str.split(' ').str[1]
apt_rent['동'] = apt_rent['시군구'].str.split(' ').str[2]
# 전용면적을 평 변환
apt_rent['평'] = round(apt_rent['전용면적(㎡)'] / 3.03,1)
# 계약년월 -> 계약년, 계약월로 분리
apt_rent['계약년'] = apt_rent['계약년월'] // 100
apt_rent['계약월'] = apt_rent['계약년월'] % 100
# 평정보를 이용하여 평형정보 생성
# 10평이하, 10평대, 20평대, 30평대, 40평대, 40평이상
apt_rent['평형']=pd.cut(
    apt_rent['평'],
    bins=[0,10,20,30,40,50,1000],
    labels='10평이하,10평대,20평대,30평대,40평대,40평이상'.split(',')
)
cond_전세 = apt_rent['구'].isin(['강남구','송파구','서초구'])
강남3구_전세 = apt_rent[cond_전세].groupby(['구','평형'])[['전세금_억원']].mean()
그외22구_전세 = apt_rent[~cond_전세].groupby(['구','평형'])[['전세금_억원']].mean()
전세종합=pd.concat([강남3구_전세,그외22구_전세])
전세종합

In [ ]:
pd.concat([매매종합,전세종합],axis=1)